In [1]:
import datetime
import numpy as np
import pandas as pd
from scipy import sparse
from collections import Counter
import time
import os
import pickle

#查看user_info的基本情况
user_info = pd.read_csv('D:\\数据挖掘\data_format1\\user_info_format1.csv')
print(user_info.info())
print(user_info)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424170 entries, 0 to 424169
Data columns (total 3 columns):
user_id      424170 non-null int64
age_range    421953 non-null float64
gender       417734 non-null float64
dtypes: float64(2), int64(1)
memory usage: 9.7 MB
None
        user_id  age_range  gender
0        376517        6.0     1.0
1        234512        5.0     0.0
2        344532        5.0     0.0
3        186135        5.0     0.0
4         30230        5.0     0.0
...         ...        ...     ...
424165   395814        3.0     1.0
424166   245950        0.0     1.0
424167   208016        NaN     NaN
424168   272535        6.0     1.0
424169    18031        3.0     1.0

[424170 rows x 3 columns]


In [2]:
#统计user_info中每个属性有多少空值
user_info.isnull().sum()

user_id         0
age_range    2217
gender       6436
dtype: int64

In [3]:
#查看user_log的基本情况，统计user_log中每个属性有多少空值
user_log = pd.read_csv('D:\\数据挖掘\data_format1\\user_log_format1.csv')
print(user_log.info())
print(user_log)
user_log.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54925330 entries, 0 to 54925329
Data columns (total 7 columns):
user_id        int64
item_id        int64
cat_id         int64
seller_id      int64
brand_id       float64
time_stamp     int64
action_type    int64
dtypes: float64(1), int64(6)
memory usage: 2.9 GB
None
          user_id  item_id  cat_id  seller_id  brand_id  time_stamp  \
0          328862   323294     833       2882    2661.0         829   
1          328862   844400    1271       2882    2661.0         829   
2          328862   575153    1271       2882    2661.0         829   
3          328862   996875    1271       2882    2661.0         829   
4          328862  1086186    1271       1253    1049.0         829   
...           ...      ...     ...        ...       ...         ...   
54925325   208016   107662     898       1346    7995.0        1110   
54925326   208016  1058313     898       1346    7995.0        1110   
54925327   208016   449814     898        9

user_id            0
item_id            0
cat_id             0
seller_id          0
brand_id       91015
time_stamp         0
action_type        0
dtype: int64

In [4]:
#去重（结果显示并无重复），检查是否有全为null的列并删去
user_info = user_info.drop_duplicates()
user_info = user_info.dropna(axis=0,how='all')
user_info

,user_id,age_range,gender
0,376517,6.0,1.0
1,234512,5.0,0.0
2,344532,5.0,0.0
3,186135,5.0,0.0
4,30230,5.0,0.0
...,...,...,...
424165,395814,3.0,1.0
424166,245950,0.0,1.0
424167,208016,NaN,NaN
424168,272535,6.0,1.0


In [5]:
#检查user_log中是否有全为null的行并删去
#user_log = user_log.rename(columns={'seller_id': 'merchant_id'})
user_log = user_log.dropna(axis=0,how='all')
user_log

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type
0,328862,323294,833,2882,2661.0,829,0
1,328862,844400,1271,2882,2661.0,829,0
2,328862,575153,1271,2882,2661.0,829,0
3,328862,996875,1271,2882,2661.0,829,0
4,328862,1086186,1271,1253,1049.0,829,0
...,...,...,...,...,...,...,...
54925325,208016,107662,898,1346,7995.0,1110,0
54925326,208016,1058313,898,1346,7995.0,1110,0
54925327,208016,449814,898,983,7995.0,1110,0
54925328,208016,634856,898,1346,7995.0,1110,0


In [6]:
user_info.user_id.value_counts()#统计user_info有多少个用户

2047      1
398440    1
310333    1
304190    1
306239    1
         ..
354808    1
352761    1
358906    1
356859    1
2049      1
Name: user_id, Length: 424170, dtype: int64

In [7]:
user_log.user_id.value_counts()#统计user_log有多少个用户

254263    14468
276887    11856
109251     9173
23106      8370
179074     8161
          ...  
234543        2
45862         2
23971         2
230975        2
109393        2
Name: user_id, Length: 424170, dtype: int64

In [8]:
print(user_log.seller_id.value_counts())#统计user_log中有多少商户
print(user_log.item_id.value_counts())#统计有多少商品
print(user_log.cat_id.value_counts())#统计有多少类别
print(user_log.brand_id.value_counts())#统计有多少品牌

3760    743217
3828    730455
184     613811
1102    541374
4173    528755
         ...  
126        231
4103       210
4449       198
4215       153
2979       143
Name: seller_id, Length: 4995, dtype: int64
67897      345905
783997     178005
636863      82480
631714      42771
61518       34801
            ...  
1020013         1
181646          1
385939          1
345470          1
635280          1
Name: item_id, Length: 1090390, dtype: int64
662     4339025
737     1749753
1505    1724239
389     1621736
656     1538969
         ...   
1364          1
920           1
572           1
1012          1
1182          1
Name: cat_id, Length: 1658, dtype: int64
3738.0    763345
1360.0    737545
1446.0    729555
1214.0    541075
5376.0    528003
           ...  
7257.0         1
2163.0         1
5415.0         1
2149.0         1
3453.0         1
Name: brand_id, Length: 8443, dtype: int64


In [9]:
user_log.action_type.value_counts()#统计点击、加购、收藏、购买各有多少次

0    48550713
2     3292144
3     3005723
1       76750
Name: action_type, dtype: int64

In [10]:
user_log_1111 = user_log[user_log.time_stamp==1111]#统计双十一当天有多少条记录
user_log_1111

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type
171,328862,406349,1280,2700,5476.0,1111,0
172,328862,406349,1280,2700,5476.0,1111,0
173,328862,807126,1181,1963,6109.0,1111,0
174,328862,406349,1280,2700,5476.0,1111,2
175,328862,406349,1280,2700,5476.0,1111,0
...,...,...,...,...,...,...,...
54925291,208016,107662,898,1346,7995.0,1111,0
54925292,208016,51712,898,3763,8434.0,1111,0
54925293,208016,107662,898,1346,7995.0,1111,0
54925294,208016,484085,955,4577,2841.0,1111,0


In [11]:
user_log_1111.action_type.value_counts()#统计双十一当天各种行为的次数

0    9188104
2    1223354
3     156450
1      14725
Name: action_type, dtype: int64

In [12]:
#统计双十一之前每天各种行为的平均次数
user_log_1110 = user_log[user_log.time_stamp<1111]
#print(user_log_1110)
#min_date=min(user_log.time_stamp)
#min_date
user_log_1110.action_type.value_counts()/184


0    213926.972826
3     15485.179348
2     11243.423913
1       337.092391
Name: action_type, dtype: float64

In [13]:
#数据清洗、预处理
user_info.age_range.fillna(0,inplace=True)#年龄缺失值用0填充，表示不知道
#numeric_index = user_info[user_info.age_range == 0].index
#user_info.loc[numeric_index,'age_range']=user_info.age_range.mode()[0]
user_info.gender.fillna(user_info.gender.mode()[0],inplace=True)# 性别用众数填充
numeric_index = user_info[user_info.gender == 2].index
user_info.loc[numeric_index,'gender']=user_info.gender.mode()[0]
print('Check any missing value?\n',user_info.isnull().any())# 检查是否有缺失值

Check any missing value?
 user_id      False
age_range    False
gender       False
dtype: bool


In [14]:
#user_log = pd.read_csv('D:\\数据挖掘\data_format1\\user_log_format1.csv')
print('Is null? \n',user_log.isnull().sum())
 
## 对brand_id缺失值进行处理
#  brand_id使用所在seller_id对应的brand_id的众数填充
missingIndex = user_log[user_log.brand_id.isnull()].index
## 思路：找到所有商店所拥有brand_id的众数，并对所缺失的brand_id与其相对应的商店进行填充
sellerMode = user_log.groupby(['seller_id']).apply(lambda x:x.brand_id.mode()[0]).reset_index()
pickUP = user_log.loc[missingIndex]
pickUP = pd.merge(pickUP,sellerMode,how='left',on=['seller_id'])[0].astype('float32')
pickUP.index = missingIndex
user_log.loc[missingIndex,'brand_id'] = pickUP
del pickUP,sellerMode,missingIndex
print('--------------------')
print('Is null? \n',user_log.isnull().sum())

Is null? 
 user_id            0
item_id            0
cat_id             0
seller_id          0
brand_id       91015
time_stamp         0
action_type        0
dtype: int64
--------------------
Is null? 
 user_id        0
item_id        0
cat_id         0
seller_id      0
brand_id       0
time_stamp     0
action_type    0
dtype: int64


In [16]:
#特征提取
# 用户基本信息：年龄，性别（类别型特征）
df_age = pd.get_dummies(user_info.age_range,prefix='age')# 对age进行哑编码
#df_sex = pd.get_dummies(user_info.gender)# 对gender进行哑编码并改变列名
#df_sex.rename(columns={0:'female',1:'male',2:'unknown'},inplace=True)
#user_info.drop("age_range", axis=1,inplace=True)
#user_info = pd.merge(user_info,df_age,how='left',on=['user_id'])
user_info = pd.concat([user_info.user_id, df_age,user_info.gender], axis=1)# 整合user信息
del df_age
print(user_info.info())
print(user_info)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 424170 entries, 0 to 424169
Data columns (total 11 columns):
user_id    424170 non-null int64
age_0.0    424170 non-null uint8
age_1.0    424170 non-null uint8
age_2.0    424170 non-null uint8
age_3.0    424170 non-null uint8
age_4.0    424170 non-null uint8
age_5.0    424170 non-null uint8
age_6.0    424170 non-null uint8
age_7.0    424170 non-null uint8
age_8.0    424170 non-null uint8
gender     424170 non-null float64
dtypes: float64(1), int64(1), uint8(9)
memory usage: 33.3 MB
None
        user_id  age_0.0  age_1.0  age_2.0  age_3.0  age_4.0  age_5.0  \
0        376517        0        0        0        0        0        0   
1        234512        0        0        0        0        0        1   
2        344532        0        0        0        0        0        1   
3        186135        0        0        0        0        0        1   
4         30230        0        0        0        0        0        1   
...         ...     

In [17]:
# 提取全部的原始行为数据...
totalActions = user_log[["user_id","action_type"]]
#totalActions.head()
#totalActions
# 对行为类别进行哑编码，0 表示点击，1 表示加入购物车, 2 表示购买，3 表示收藏.
df = pd.get_dummies(totalActions['action_type'],prefix='userTotalAction')
# 统计日志行为中用户点击、加购、购买、收藏的总次数
totalActions = pd.concat([totalActions.user_id, df], axis=1).groupby(['user_id'], as_index=False).sum()
totalActions['userTotalAction'] = totalActions['userTotalAction_0']+totalActions['userTotalAction_1']+totalActions['userTotalAction_2']+totalActions['userTotalAction_3']
del df
totalActions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 424170 entries, 0 to 424169
Data columns (total 6 columns):
user_id              424170 non-null int64
userTotalAction_0    424170 non-null float64
userTotalAction_1    424170 non-null float64
userTotalAction_2    424170 non-null float64
userTotalAction_3    424170 non-null float64
userTotalAction      424170 non-null float64
dtypes: float64(5), int64(1)
memory usage: 22.7 MB


In [18]:
#用户交互次数在所有用户中的地位
print('所有用户交互次数：'+str(user_log.shape[0]))
print('所有用户数：'+str(user_log['user_id'].nunique()))
print('所有用户平均交互次数：'+str(user_log.shape[0]/user_log['user_id'].nunique()))
totalActions['userTotalActionRatio'] = totalActions['userTotalAction']/user_log.shape[0]#用户交互次数占所有交互次数的比例
totalActions['userTotalActionDiff'] = totalActions['userTotalAction']-user_log.shape[0]/user_log['user_id'].nunique()#用户交互次数与平均交互次数的差值

所有用户交互次数：54925330
所有用户数：424170
所有用户平均交互次数：129.48895490015795


In [19]:
#用户点击次数在所有用户点击次数中的地位
print('所有用户点击次数：'+str(user_log[user_log.action_type==0].shape[0]))
totalActions['userClickRatio'] = totalActions['userTotalAction_0']/user_log[user_log.action_type==0].shape[0]#用户点击次数占所有点击次数的比例
print('用户平均点击次数：'+str(user_log[user_log.action_type==0].shape[0]/user_log['user_id'].nunique()))
totalActions['userClickDiff'] = totalActions['userTotalAction_0']-user_log[user_log.action_type==0].shape[0]/user_log['user_id'].nunique()#用户点击次数与平均点击次数的差值

所有用户点击次数：48550713
用户平均点击次数：114.46050640073555


In [20]:
#用户加入购物车次数在所有用户加入购物车次数中的地位
print('所有用户加入购物车次数：'+str(user_log[user_log.action_type==1].shape[0]))
totalActions['userAddRatio'] = totalActions['userTotalAction_1']/user_log[user_log.action_type==1].shape[0]#用户加入购物车次数占所有加入购物车次数的比例
print('用户平均加入购物车次数：'+str(user_log[user_log.action_type==1].shape[0]/user_log['user_id'].nunique()))
totalActions['userAddDiff'] = totalActions['userTotalAction_1']-user_log[user_log.action_type==1].shape[0]/user_log['user_id'].nunique()#用户加入购物车次数与平均加入购物车次数差值

所有用户加入购物车次数：76750
用户平均加入购物车次数：0.18094160360232925


In [21]:
#用户购买次数在所有用户购买次数中的地位
print('所有用户购买次数：'+str(user_log[user_log.action_type==2].shape[0]))
totalActions['userBuyRatio'] = totalActions['userTotalAction_2']/user_log[user_log.action_type==2].shape[0]#用户购买次数占所有购买次数的比例
print('用户平均购买次数：'+str(user_log[user_log.action_type==2].shape[0]/user_log['user_id'].nunique()))
totalActions['userBuyDiff'] = totalActions['userTotalAction_2']-user_log[user_log.action_type==2].shape[0]/user_log['user_id'].nunique()#用户购买次数与平均购买次数的差值

所有用户购买次数：3292144
用户平均购买次数：7.761378692505364


In [22]:
#用户收藏次数在所有用户收藏次数中的地位
print('所有用户收藏次数：'+str(user_log[user_log.action_type==3].shape[0]))
totalActions['userSaveRatio'] = totalActions['userTotalAction_3']/user_log[user_log.action_type==3].shape[0]#用户收藏次数占所有收藏次数的比例
print('用户平均收藏次数：'+str(user_log[user_log.action_type==3].shape[0]/user_log['user_id'].nunique()))
totalActions['userSaveDiff'] = totalActions['userTotalAction_3']-user_log[user_log.action_type==3].shape[0]/user_log['user_id'].nunique()#用户收藏次数与平均收藏次数的差值

所有用户收藏次数：3005723
用户平均收藏次数：7.086128203314709


In [23]:
# 统计用户点击，加购，收藏，购买次数占用户总交互次数的比例
totalActions['userClick_ratio'] = totalActions['userTotalAction_0']/totalActions['userTotalAction']
totalActions['userAdd_ratio'] = totalActions['userTotalAction_1']/totalActions['userTotalAction']
totalActions['userBuy_ratio'] = totalActions['userTotalAction_2']/totalActions['userTotalAction']
totalActions['userSave_ratio'] = totalActions['userTotalAction_3']/totalActions['userTotalAction']

In [24]:
# 统计日志行为中用户的点击、加购、收藏的购买转化率
totalActions['userTotalAction_0_ratio'] = np.log1p(totalActions['userTotalAction_2']) - np.log1p(totalActions['userTotalAction_0'])#用户的点击购买转化率
totalActions['userTotalAction_0_ratio_diff'] = totalActions['userTotalAction_0_ratio'] - totalActions['userTotalAction_0_ratio'].mean()#用户的点击购买转化率与平均购买转化率的差值
totalActions['userTotalAction_1_ratio'] = np.log1p(totalActions['userTotalAction_2']) - np.log1p(totalActions['userTotalAction_1'])#用户的加购购买转化率
totalActions['userTotalAction_1_ratio_diff'] = totalActions['userTotalAction_1_ratio'] - totalActions['userTotalAction_1_ratio'].mean()#用户的加购购买转化率与平均加购购买转化率的差值
totalActions['userTotalAction_3_ratio'] = np.log1p(totalActions['userTotalAction_2']) - np.log1p(totalActions['userTotalAction_3'])#用户的收藏购买转化率
totalActions['userTotalAction_3_ratio_diff'] = totalActions['userTotalAction_3_ratio'] - totalActions['userTotalAction_3_ratio'].mean()#用户的收藏购买转化率与平均收藏购买转化率的差值
totalActions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 424170 entries, 0 to 424169
Data columns (total 26 columns):
user_id                         424170 non-null int64
userTotalAction_0               424170 non-null float64
userTotalAction_1               424170 non-null float64
userTotalAction_2               424170 non-null float64
userTotalAction_3               424170 non-null float64
userTotalAction                 424170 non-null float64
userTotalActionRatio            424170 non-null float64
userTotalActionDiff             424170 non-null float64
userClickRatio                  424170 non-null float64
userClickDiff                   424170 non-null float64
userAddRatio                    424170 non-null float64
userAddDiff                     424170 non-null float64
userBuyRatio                    424170 non-null float64
userBuyDiff                     424170 non-null float64
userSaveRatio                   424170 non-null float64
userSaveDiff                    424170 non-null flo

In [25]:
# 对数值型特征手动标准化
numeric_cols = totalActions.columns[totalActions.dtypes == 'float64']
#print(numeric_cols)
numeric_col_means = totalActions.loc[:, numeric_cols].mean()
numeric_col_std = totalActions.loc[:, numeric_cols].std()
totalActions.loc[:, numeric_cols] = (totalActions.loc[:, numeric_cols] - numeric_col_means) / numeric_col_std
totalActions.head(5)

,user_id,userTotalAction_0,userTotalAction_1,userTotalAction_2,userTotalAction_3,userTotalAction,userTotalActionRatio,userTotalActionDiff,userClickRatio,userClickDiff,...,userClick_ratio,userAdd_ratio,userBuy_ratio,userSave_ratio,userTotalAction_0_ratio,userTotalAction_0_ratio_diff,userTotalAction_1_ratio,userTotalAction_1_ratio_diff,userTotalAction_3_ratio,userTotalAction_3_ratio_diff
0,1,-0.505625,-0.204257,-0.222497,-0.355284,-0.513955,-0.513955,-0.513955,-0.505625,-0.505625,...,-0.248463,-0.184557,0.960101,-0.548872,0.993740,0.993740,0.160741,0.160741,0.879081,0.879081
1,2,-0.390001,-0.204257,0.788060,-0.255008,-0.354158,-0.354158,-0.354158,-0.390001,-0.390001,...,-0.822513,-0.184557,1.423089,-0.199305,1.243542,1.243542,1.127229,1.127229,0.613294,0.613294
2,3,-0.297502,-0.204257,-0.475136,-0.305146,-0.327525,-0.327525,-0.327525,-0.297502,-0.297502,...,0.613120,-0.184557,-0.449287,-0.386941,-0.308369,-0.308369,-0.265947,-0.265947,0.065761,0.065761
3,4,-0.378439,-0.204257,-0.854095,-0.355284,-0.423403,-0.423403,-0.423403,-0.378439,-0.378439,...,1.039019,-0.184557,-0.894163,-0.548872,-1.057774,-1.057774,-1.427917,-1.427917,-0.110505,-0.110505
4,5,0.205460,-0.204257,0.661741,0.146095,0.231765,0.231765,0.231765,0.205460,0.205460,...,0.140366,-0.184557,-0.262266,0.087622,-0.116691,-0.116691,1.039737,1.039737,-0.467538,-0.467538


In [26]:
# 将统计好的数量和转化率进行拼接
user_info = pd.merge(user_info,totalActions,how='left',on=['user_id'])
del totalActions
user_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 424170 entries, 0 to 424169
Data columns (total 36 columns):
user_id                         424170 non-null int64
age_0.0                         424170 non-null uint8
age_1.0                         424170 non-null uint8
age_2.0                         424170 non-null uint8
age_3.0                         424170 non-null uint8
age_4.0                         424170 non-null uint8
age_5.0                         424170 non-null uint8
age_6.0                         424170 non-null uint8
age_7.0                         424170 non-null uint8
age_8.0                         424170 non-null uint8
gender                          424170 non-null float64
userTotalAction_0               424170 non-null float64
userTotalAction_1               424170 non-null float64
userTotalAction_2               424170 non-null float64
userTotalAction_3               424170 non-null float64
userTotalAction                 424170 non-null float64
userTotalActi

In [27]:
# 用户六个月中做出行为的不同商品种类数目
item_cnt = user_log.groupby(['user_id'])['item_id'].nunique()
# 用户六个月中做出行为的不同种类数量
cat_cnt = user_log.groupby(['user_id'])['cat_id'].nunique()
# 用户六个月中做出行为的不同店铺数量
seller_cnt = user_log.groupby(['user_id'])['seller_id'].nunique()
# 用户六个月中做出行为的不同品牌数量
brand_cnt = user_log.groupby(['user_id'])['brand_id'].nunique()
# 用户六个月中做出行为的天数
days_cnt = user_log.groupby(['user_id'])['time_stamp'].nunique()

typeCount_result = pd.concat([item_cnt,cat_cnt],axis=1)
typeCount_result = pd.concat([typeCount_result,seller_cnt],axis=1)
typeCount_result = pd.concat([typeCount_result,brand_cnt],axis=1)
typeCount_result = pd.concat([typeCount_result,days_cnt],axis=1)
typeCount_result.rename(columns={'item_id':'item_cnt','cat_id':'cat_cnt','seller_id':'seller_cnt','brand_id':'brand_counts','time_stamp':'active_days'},inplace=True)
typeCount_result.reset_index(inplace=True)
typeCount_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424170 entries, 0 to 424169
Data columns (total 6 columns):
user_id         424170 non-null int64
item_cnt        424170 non-null int64
cat_cnt         424170 non-null int64
seller_cnt      424170 non-null int64
brand_counts    424170 non-null int64
active_days     424170 non-null int64
dtypes: int64(6)
memory usage: 19.4 MB


In [28]:
# 对数值型特征手动标准化
numeric_cols = typeCount_result.columns[typeCount_result.dtypes == 'int64']
print(numeric_cols)
numeric_cols=numeric_cols[1:6]#去掉user_id这一列
print(numeric_cols)
numeric_col_means = typeCount_result.loc[:, numeric_cols].mean()
numeric_col_std = typeCount_result.loc[:, numeric_cols].std()
typeCount_result.loc[:, numeric_cols] = (typeCount_result.loc[:, numeric_cols] - numeric_col_means) / numeric_col_std
typeCount_result.head(5)

Index(['user_id', 'item_cnt', 'cat_cnt', 'seller_cnt', 'brand_counts',
       'active_days'],
      dtype='object')
Index(['item_cnt', 'cat_cnt', 'seller_cnt', 'brand_counts', 'active_days'], dtype='object')


,user_id,item_cnt,cat_cnt,seller_cnt,brand_counts,active_days
0,1,-0.619293,-0.985184,-0.713461,-0.721148,-0.839753
1,2,-0.315566,-0.507574,-0.565709,-0.539438,-0.557186
2,3,-0.295971,-0.209068,-0.299757,-0.327442,-0.274619
3,4,-0.462530,-0.567275,-0.624810,-0.630293,-0.486544
4,5,0.115530,1.044659,0.675404,0.793106,0.926290


In [29]:
## 将统计好的数量进行拼接
user_info = pd.merge(user_info,typeCount_result,how='left',on=['user_id'])
del typeCount_result
user_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 424170 entries, 0 to 424169
Data columns (total 41 columns):
user_id                         424170 non-null int64
age_0.0                         424170 non-null uint8
age_1.0                         424170 non-null uint8
age_2.0                         424170 non-null uint8
age_3.0                         424170 non-null uint8
age_4.0                         424170 non-null uint8
age_5.0                         424170 non-null uint8
age_6.0                         424170 non-null uint8
age_7.0                         424170 non-null uint8
age_8.0                         424170 non-null uint8
gender                          424170 non-null float64
userTotalAction_0               424170 non-null float64
userTotalAction_1               424170 non-null float64
userTotalAction_2               424170 non-null float64
userTotalAction_3               424170 non-null float64
userTotalAction                 424170 non-null float64
userTotalActi

In [30]:
## 统计双十一之前，用户重复购买过的商家数量
#先填充，再进行标准化处理
### --------------------------------------------------------------------------
repeatSellerCount = user_log[["user_id","seller_id","time_stamp","action_type"]]
repeatSellerCount = repeatSellerCount[(repeatSellerCount.action_type == 2) & (repeatSellerCount.time_stamp < 1111)]
repeatSellerCount.drop_duplicates(inplace=True)
repeatSellerCount = repeatSellerCount.groupby(['user_id','seller_id'])['time_stamp'].count().reset_index()
repeatSellerCount = repeatSellerCount[repeatSellerCount.time_stamp > 1]
repeatSellerCount = repeatSellerCount.groupby(['user_id'])['seller_id'].count().reset_index()
repeatSellerCount.rename(columns={'seller_id':'repeat_seller_count1'},inplace=True)
repeatSellerCount.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74765 entries, 0 to 74764
Data columns (total 2 columns):
user_id                 74765 non-null int64
repeat_seller_count1    74765 non-null int64
dtypes: int64(2)
memory usage: 1.1 MB


In [31]:
repeatSellerCount = pd.merge(user_info['user_id'],repeatSellerCount,how='left',on=['user_id'])
# 没有重复购买的user用0填充
repeatSellerCount.repeat_seller_count1.fillna(0,inplace=True)
repeatSellerCount['repeat_seller1'] = repeatSellerCount['repeat_seller_count1'].map(lambda x: 1 if x != 0 else 0)
#print(repeatSellerCount)
repeatSellerCount.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 424170 entries, 0 to 424169
Data columns (total 3 columns):
user_id                 424170 non-null int64
repeat_seller_count1    424170 non-null float64
repeat_seller1          424170 non-null int64
dtypes: float64(1), int64(2)
memory usage: 12.9 MB


In [32]:
# 对数值型特征手动标准化
numeric_cols = repeatSellerCount.columns[repeatSellerCount.dtypes == 'float64']
print(numeric_cols)
#numeric_cols=numeric_cols[1]#只对repeat_seller_count1这一列标准化
#print(numeric_cols)
numeric_col_means = repeatSellerCount.loc[:, numeric_cols].mean()
numeric_col_std = repeatSellerCount.loc[:, numeric_cols].std()
repeatSellerCount.loc[:, numeric_cols] = (repeatSellerCount.loc[:, numeric_cols] - numeric_col_means) / numeric_col_std
repeatSellerCount.head(5)

Index(['repeat_seller_count1'], dtype='object')


,user_id,repeat_seller_count1,repeat_seller1
0,376517,-0.347519,0
1,234512,-0.347519,0
2,344532,1.102240,1
3,186135,-0.347519,0
4,30230,-0.347519,0


In [33]:
user_info = pd.merge(user_info,repeatSellerCount,how='left',on=['user_id'])
del repeatSellerCount
user_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 424170 entries, 0 to 424169
Data columns (total 43 columns):
user_id                         424170 non-null int64
age_0.0                         424170 non-null uint8
age_1.0                         424170 non-null uint8
age_2.0                         424170 non-null uint8
age_3.0                         424170 non-null uint8
age_4.0                         424170 non-null uint8
age_5.0                         424170 non-null uint8
age_6.0                         424170 non-null uint8
age_7.0                         424170 non-null uint8
age_8.0                         424170 non-null uint8
gender                          424170 non-null float64
userTotalAction_0               424170 non-null float64
userTotalAction_1               424170 non-null float64
userTotalAction_2               424170 non-null float64
userTotalAction_3               424170 non-null float64
userTotalAction                 424170 non-null float64
userTotalActi

In [35]:
# 用户交互的间隔
# 统计每月的点击次数，每月的加入购物次数，每月的购买次数，每月的收藏次数
### --------------------------------------------------------------------------
monthActionsCount = user_log[["user_id","time_stamp","action_type"]]
result = list()
for i in range(5,12):
    start = int(str(i)+'00')
    end = int(str(i)+'30')
    # 获取i月的数据
    example = monthActionsCount[(monthActionsCount.time_stamp >= start) & (monthActionsCount.time_stamp < end)]
    # 对i月的交互行为进行哑编码
    df = pd.get_dummies(example['action_type'],prefix='%d_Action'%i)
    df[str(i)+'_Action'] = df[str(i)+'_Action_0']+df[str(i)+'_Action_1']+df[str(i)+'_Action_2']+df[str(i)+'_Action_3']
    # 将example的time_stamp设为月份值（5,6，。。。，11）
    example.loc[:,'time_stamp'] = example.time_stamp.apply(lambda x: int(str(x)[0]) if len(str(x)) == 3 else int(str(x)[:2]))
    result.append(pd.concat([example, df], axis=1).groupby(['user_id','time_stamp'],as_index=False).sum())

#print(result)    
for i in range(0,7):
    user_info = pd.merge(user_info,result[i],how='left',on=['user_id'])
    user_info.fillna(0,inplace=True)

E:\Anaconda3\lib\site-packages\pandas\core\indexing.py:576: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [36]:
for col in ['time_stamp_x','action_type_x','time_stamp_y','action_type_y','time_stamp','action_type']:
    del user_info[col]
for i in range(5,12):
    user_info[str(i)+'_Action'] = user_info[str(i)+'_Action_0']+user_info[str(i)+'_Action_1']+user_info[str(i)+'_Action_2']+user_info[str(i)+'_Action_3']

In [37]:
user_info.info()
print(user_info.columns[43:])

<class 'pandas.core.frame.DataFrame'>
Int64Index: 424170 entries, 0 to 424169
Data columns (total 78 columns):
user_id                         424170 non-null int64
age_0.0                         424170 non-null uint8
age_1.0                         424170 non-null uint8
age_2.0                         424170 non-null uint8
age_3.0                         424170 non-null uint8
age_4.0                         424170 non-null uint8
age_5.0                         424170 non-null uint8
age_6.0                         424170 non-null uint8
age_7.0                         424170 non-null uint8
age_8.0                         424170 non-null uint8
gender                          424170 non-null float64
userTotalAction_0               424170 non-null float64
userTotalAction_1               424170 non-null float64
userTotalAction_2               424170 non-null float64
userTotalAction_3               424170 non-null float64
userTotalAction                 424170 non-null float64
userTotalActi

In [38]:
# 对数值型特征手动标准化
numeric_cols = user_info.columns[43:]
#print(numeric_cols)
numeric_col_means = user_info.loc[:, numeric_cols].mean()
numeric_col_std = user_info.loc[:, numeric_cols].std()
user_info.loc[:, numeric_cols] = (user_info.loc[:, numeric_cols] - numeric_col_means) / numeric_col_std
user_info.head(5)

,user_id,age_0.0,age_1.0,age_2.0,age_3.0,age_4.0,age_5.0,age_6.0,age_7.0,age_8.0,...,10_Action_0,10_Action_1,10_Action_2,10_Action_3,10_Action,11_Action_0,11_Action_1,11_Action_2,11_Action_3,11_Action
0,376517,0,0,0,0,0,0,1,0,0,...,0.128608,-0.095484,1.032893,-0.277882,0.137327,-0.229312,-0.174409,-0.419134,-0.359062,-0.265275
1,234512,0,0,0,0,0,1,0,0,0,...,-0.260988,-0.095484,0.014132,-0.277882,-0.272109,-0.199201,-0.174409,0.636380,-0.004993,-0.165281
2,344532,0,0,0,0,0,1,0,0,0,...,-0.171082,-0.095484,1.542273,0.160268,-0.053743,-0.304589,-0.174409,-0.419134,0.880181,-0.236705
3,186135,0,0,0,0,0,1,0,0,0,...,-0.470771,-0.095484,-0.495248,-0.277882,-0.490475,2.254841,-0.174409,-0.419134,0.526112,2.163151
4,30230,0,0,0,0,0,1,0,0,0,...,-0.320926,-0.095484,0.014132,-0.277882,-0.326700,-0.409978,-0.174409,-0.419134,-0.359062,-0.436694


In [39]:
#保存用户特征
filePath='D:\\数据挖掘\data_format1\\userInfo_Features_3.pkl'
pickle.dump(user_info, open(filePath, 'wb'))
user_info.to_csv('D:\\数据挖掘\data_format1\\user_info_format_feature_3.csv')

In [40]:
#商户相关特征
# 统计每个商户的商品，种类，品牌总数，并放入dataFrame[seller_id,xx_number]为列名，便于往后的拼接
# （表示商户的规模大小）
itemNumber = user_log[['seller_id','item_id']].groupby(['seller_id'])['item_id'].nunique().reset_index()
catNumber = user_log[['seller_id','cat_id']].groupby(['seller_id'])['cat_id'].nunique().reset_index()
brandNumber = user_log[['seller_id','brand_id']].groupby(['seller_id'])['brand_id'].nunique().reset_index()
itemNumber.rename(columns={'item_id':'item_number'},inplace=True)
catNumber.rename(columns={'cat_id':'cat_number'},inplace=True)
brandNumber.rename(columns={'brand_id':'brand_number'},inplace=True)

In [41]:
##统计被点击，被加入购物车，被购买，被收藏次数
###统计被点击购买转化率，被加入购物车购买转化率，被收藏次数购买转化率
sellerTotalActions = user_log[["seller_id","action_type"]]
df = pd.get_dummies(sellerTotalActions['action_type'],prefix='sellerTotalAction')
sellerTotalActions = pd.concat([sellerTotalActions, df], axis=1).groupby(['seller_id'], as_index=False).sum()
sellerTotalActions.drop("action_type", axis=1,inplace=True)
sellerTotalActions['sellerTotalAction'] = sellerTotalActions['sellerTotalAction_0']+sellerTotalActions['sellerTotalAction_1']+sellerTotalActions['sellerTotalAction_2']+sellerTotalActions['sellerTotalAction_3']
del df
#　构造转化率字段
sellerTotalActions['sellerTotalAction_0_ratio'] = np.log1p(sellerTotalActions['sellerTotalAction_2']) - np.log1p(sellerTotalActions['sellerTotalAction_0'])
sellerTotalActions['sellerTotalAction_1_ratio'] = np.log1p(sellerTotalActions['sellerTotalAction_2']) - np.log1p(sellerTotalActions['sellerTotalAction_1'])
sellerTotalActions['sellerTotalAction_3_ratio'] = np.log1p(sellerTotalActions['sellerTotalAction_2']) - np.log1p(sellerTotalActions['sellerTotalAction_3'])
sellerTotalActions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4995 entries, 0 to 4994
Data columns (total 9 columns):
seller_id                    4995 non-null int64
sellerTotalAction_0          4995 non-null float64
sellerTotalAction_1          4995 non-null float64
sellerTotalAction_2          4995 non-null float64
sellerTotalAction_3          4995 non-null float64
sellerTotalAction            4995 non-null float64
sellerTotalAction_0_ratio    4995 non-null float64
sellerTotalAction_1_ratio    4995 non-null float64
sellerTotalAction_3_ratio    4995 non-null float64
dtypes: float64(8), int64(1)
memory usage: 390.2 KB


In [42]:
# 将统计好的数量和转化率进行拼接
sellers = pd.merge(sellerTotalActions,itemNumber,on=['seller_id'])
sellers = pd.merge(sellers,catNumber,on=['seller_id'])
sellers = pd.merge(sellers,brandNumber,on=['seller_id'])
del sellerTotalActions,itemNumber,catNumber,brandNumber
sellers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4995 entries, 0 to 4994
Data columns (total 12 columns):
seller_id                    4995 non-null int64
sellerTotalAction_0          4995 non-null float64
sellerTotalAction_1          4995 non-null float64
sellerTotalAction_2          4995 non-null float64
sellerTotalAction_3          4995 non-null float64
sellerTotalAction            4995 non-null float64
sellerTotalAction_0_ratio    4995 non-null float64
sellerTotalAction_1_ratio    4995 non-null float64
sellerTotalAction_3_ratio    4995 non-null float64
item_number                  4995 non-null int64
cat_number                   4995 non-null int64
brand_number                 4995 non-null int64
dtypes: float64(8), int64(4)
memory usage: 507.3 KB


In [43]:
# 统计每个商户的商品数，商品种类、品牌占总量的比例（表示商户的规模大小）
sellers['item_ratio'] = sellers['item_number']/user_log['item_id'].nunique()
sellers['cat_ratio'] = sellers['cat_number']/user_log['cat_id'].nunique()
sellers['brand_ratio'] = sellers['brand_number']/user_log['brand_id'].nunique()

In [44]:
#统计每个商户被点击的人数，被加入购物车的人数，被购买的人数，被收藏的人数
peoCount = user_log[["user_id","seller_id","action_type"]]
df = pd.get_dummies(peoCount['action_type'],prefix='seller_peopleNumber')
peoCount = pd.concat([peoCount, df], axis=1)
peoCount.drop("action_type", axis=1,inplace=True)
peoCount.drop_duplicates(inplace=True)
df1 = peoCount.groupby(['seller_id']).apply(lambda x:x.seller_peopleNumber_0.sum())
df2 = peoCount.groupby(['seller_id']).apply(lambda x:x.seller_peopleNumber_1.sum())
df3 = peoCount.groupby(['seller_id']).apply(lambda x:x.seller_peopleNumber_2.sum())
df4 = peoCount.groupby(['seller_id']).apply(lambda x:x.seller_peopleNumber_3.sum())
peoCount = pd.concat([df1, df2,df3, df4], axis=1).reset_index()
del df1,df2,df3,df4
peoCount.rename(columns={0:'seller_peopleNum_0',1:'seller_peopleNum_1',2:'seller_peopleNum_2',3:'seller_peopleNum_3'},inplace=True)
peoCount.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4995 entries, 0 to 4994
Data columns (total 5 columns):
seller_id             4995 non-null int64
seller_peopleNum_0    4995 non-null int64
seller_peopleNum_1    4995 non-null int64
seller_peopleNum_2    4995 non-null int64
seller_peopleNum_3    4995 non-null int64
dtypes: int64(5)
memory usage: 195.2 KB


In [45]:
sellers = pd.merge(sellers,peoCount,on=['seller_id'])
del peoCount
sellers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4995 entries, 0 to 4994
Data columns (total 19 columns):
seller_id                    4995 non-null int64
sellerTotalAction_0          4995 non-null float64
sellerTotalAction_1          4995 non-null float64
sellerTotalAction_2          4995 non-null float64
sellerTotalAction_3          4995 non-null float64
sellerTotalAction            4995 non-null float64
sellerTotalAction_0_ratio    4995 non-null float64
sellerTotalAction_1_ratio    4995 non-null float64
sellerTotalAction_3_ratio    4995 non-null float64
item_number                  4995 non-null int64
cat_number                   4995 non-null int64
brand_number                 4995 non-null int64
item_ratio                   4995 non-null float64
cat_ratio                    4995 non-null float64
brand_ratio                  4995 non-null float64
seller_peopleNum_0           4995 non-null int64
seller_peopleNum_1           4995 non-null int64
seller_peopleNum_2           4995 non-

In [46]:
# 统计每个商户被点击、加购、购买、收藏的人数占有点击、加购、购买、收藏行为人数的比例
sellers['click_people_ratio'] = sellers['seller_peopleNum_0']/user_log[user_log['action_type'] == 0]['user_id'].nunique()
sellers['add_people_ratio'] = sellers['seller_peopleNum_1']/user_log[user_log['action_type'] == 1]['user_id'].nunique()
sellers['buy_people_ratio'] = sellers['seller_peopleNum_2']/user_log[user_log['action_type'] == 2]['user_id'].nunique()
sellers['save_people_ratio'] = sellers['seller_peopleNum_3']/user_log[user_log['action_type'] == 3]['user_id'].nunique()

In [47]:
 # 统计双十一之前商户重复买家总数量（表示商户对于新用户的留存能力）
repeatPeoCount = user_log[(user_log.time_stamp < 1111) & (user_log.action_type == 2)]
repeatPeoCount = repeatPeoCount.groupby(['seller_id'])['user_id'].value_counts().to_frame()
repeatPeoCount.rename(columns={'user_id':'Buy_Number'},inplace=True)
repeatPeoCount.reset_index(inplace=True)
print(repeatPeoCount)
repeatPeoCount = repeatPeoCount[repeatPeoCount.Buy_Number > 1]
repeatPeoCount = repeatPeoCount.groupby(['seller_id']).apply(lambda x:len(x.user_id)).reset_index()
repeatPeoCount = pd.merge(pd.DataFrame({'seller_id':range(1, 4996 ,1)}),repeatPeoCount,how='left',on=['seller_id']).fillna(0)
repeatPeoCount.rename(columns={0:'repeatBuy_peopleNumber'},inplace=True)
print('--------------------')
repeatPeoCount.info()
print(repeatPeoCount)

         seller_id  user_id  Buy_Number
0                1      406          21
1                1    56832          21
2                1   180072          20
3                1   339584          15
4                1    88181          14
...            ...      ...         ...
1359013       4995   421568           1
1359014       4995   423646           1
1359015       4995   423738           1
1359016       4995   423884           1
1359017       4995   424153           1

[1359018 rows x 3 columns]
--------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 4995 entries, 0 to 4994
Data columns (total 2 columns):
seller_id                 4995 non-null int64
repeatBuy_peopleNumber    4995 non-null float64
dtypes: float64(1), int64(1)
memory usage: 117.1 KB
      seller_id  repeatBuy_peopleNumber
0             1                  2214.0
1             2                     4.0
2             3                     2.0
3             4                    48.0
4             5    

In [48]:
sellers = pd.merge(sellers,repeatPeoCount,on=['seller_id'])
del repeatPeoCount
sellers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4995 entries, 0 to 4994
Data columns (total 24 columns):
seller_id                    4995 non-null int64
sellerTotalAction_0          4995 non-null float64
sellerTotalAction_1          4995 non-null float64
sellerTotalAction_2          4995 non-null float64
sellerTotalAction_3          4995 non-null float64
sellerTotalAction            4995 non-null float64
sellerTotalAction_0_ratio    4995 non-null float64
sellerTotalAction_1_ratio    4995 non-null float64
sellerTotalAction_3_ratio    4995 non-null float64
item_number                  4995 non-null int64
cat_number                   4995 non-null int64
brand_number                 4995 non-null int64
item_ratio                   4995 non-null float64
cat_ratio                    4995 non-null float64
brand_ratio                  4995 non-null float64
seller_peopleNum_0           4995 non-null int64
seller_peopleNum_1           4995 non-null int64
seller_peopleNum_2           4995 non-

In [49]:
# 对数值型特征手动标准化
numeric_cols = sellers.columns[1:]
print(numeric_cols)
numeric_col_means = sellers.loc[:, numeric_cols].mean()
numeric_col_std = sellers.loc[:, numeric_cols].std()
sellers.loc[:, numeric_cols] = (sellers.loc[:, numeric_cols] - numeric_col_means) / numeric_col_std
sellers.head(5)

Index(['sellerTotalAction_0', 'sellerTotalAction_1', 'sellerTotalAction_2',
       'sellerTotalAction_3', 'sellerTotalAction', 'sellerTotalAction_0_ratio',
       'sellerTotalAction_1_ratio', 'sellerTotalAction_3_ratio', 'item_number',
       'cat_number', 'brand_number', 'item_ratio', 'cat_ratio', 'brand_ratio',
       'seller_peopleNum_0', 'seller_peopleNum_1', 'seller_peopleNum_2',
       'seller_peopleNum_3', 'click_people_ratio', 'add_people_ratio',
       'buy_people_ratio', 'save_people_ratio', 'repeatBuy_peopleNumber'],
      dtype='object')


,seller_id,sellerTotalAction_0,sellerTotalAction_1,sellerTotalAction_2,sellerTotalAction_3,sellerTotalAction,sellerTotalAction_0_ratio,sellerTotalAction_1_ratio,sellerTotalAction_3_ratio,item_number,...,brand_ratio,seller_peopleNum_0,seller_peopleNum_1,seller_peopleNum_2,seller_peopleNum_3,click_people_ratio,add_people_ratio,buy_people_ratio,save_people_ratio,repeatBuy_peopleNumber
0,1,10.972787,11.897055,14.249959,7.235092,11.050909,-0.388080,-0.014704,0.135857,8.107215,...,-0.167130,6.739658,9.703700,10.099788,6.360937,6.739658,9.703700,10.099788,6.360937,14.225154
1,2,-0.282662,-0.204431,-0.392982,-0.272506,-0.290466,0.317178,-0.777947,0.062929,-0.188952,...,-0.273422,-0.444305,-0.248208,-0.393341,-0.372739,-0.444305,-0.248208,-0.393341,-0.372739,-0.467010
2,3,-0.269099,-0.315454,-0.494971,-0.254051,-0.281239,-1.410094,-1.307497,-1.483019,-0.138993,...,-0.273422,-0.393370,-0.325057,-0.527563,-0.340727,-0.393370,-0.325057,-0.527563,-0.340727,-0.480306
3,4,-0.260019,-0.370965,-0.305204,-0.260600,-0.265714,0.570317,1.074886,0.456281,-0.186014,...,-0.167130,-0.322161,-0.401906,-0.337414,-0.336552,-0.322161,-0.401906,-0.337414,-0.336552,-0.174497
4,5,-0.082222,-0.176675,-0.430601,-0.027234,-0.094432,-1.959706,-1.230329,-1.982849,1.298069,...,-0.273422,0.222927,-0.132935,-0.450665,0.087956,0.222927,-0.132935,-0.450665,0.087956,-0.400530


In [51]:
#保存用户特征
filePath='D:\\数据挖掘\data_format1\\sellerInfo_Features_3.pkl'
pickle.dump(sellers, open(filePath, 'wb'))
sellers.to_csv('D:\\数据挖掘\data_format1\\seller_info_format_feature_3.csv')

In [52]:
## 提取预测目标的行为数据
trainData = pd.read_csv('D:\\数据挖掘\data_format1\\train_format1.csv')
trainData.rename(columns={'merchant_id':'seller_id'},inplace=True)
testData = pd.read_csv('D:\\数据挖掘\data_format1\\test_format1.csv')
testData.rename(columns={'merchant_id':'seller_id'},inplace=True)
targetIndex = pd.concat([trainData[['user_id', 'seller_id']],testData[['user_id', 'seller_id']]],ignore_index=True)
logs = pd.merge(targetIndex,user_log,on=['user_id', 'seller_id'])
del trainData,testData,targetIndex
logs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5662515 entries, 0 to 5662514
Data columns (total 7 columns):
user_id        int64
seller_id      int64
item_id        int64
cat_id         int64
brand_id       float64
time_stamp     int64
action_type    int64
dtypes: float64(1), int64(6)
memory usage: 345.6 MB


In [53]:
#统计logs中每个属性有多少空值
logs.isnull().sum()

user_id        0
seller_id      0
item_id        0
cat_id         0
brand_id       0
time_stamp     0
action_type    0
dtype: int64

In [54]:
### 统计用户对预测的商店的行为特征，例如点击，加入购物车，购买，收藏的总次数,以及各种转化率
df_result = logs[["user_id", "seller_id","action_type"]]
df = pd.get_dummies(df_result['action_type'],prefix='userSellerAction')
df_result = pd.concat([df_result, df], axis=1).groupby(['user_id', 'seller_id'], as_index=False).sum()
df_result['userSellerAction'] = df_result['userSellerAction_0']+df_result['userSellerAction_1']+df_result['userSellerAction_2']+df_result['userSellerAction_3']
del df
df_result.drop("action_type", axis=1,inplace=True)
df_result['userSellerAction_0_ratio'] = np.log1p(df_result['userSellerAction_2']) - np.log1p(df_result['userSellerAction_0'])
df_result['userSellerAction_1_ratio'] = np.log1p(df_result['userSellerAction_2']) - np.log1p(df_result['userSellerAction_1'])
df_result['userSellerAction_3_ratio'] = np.log1p(df_result['userSellerAction_2']) - np.log1p(df_result['userSellerAction_3'])
df_result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 522341 entries, 0 to 522340
Data columns (total 10 columns):
user_id                     522341 non-null int64
seller_id                   522341 non-null int64
userSellerAction_0          522341 non-null float64
userSellerAction_1          522341 non-null float64
userSellerAction_2          522341 non-null float64
userSellerAction_3          522341 non-null float64
userSellerAction            522341 non-null float64
userSellerAction_0_ratio    522341 non-null float64
userSellerAction_1_ratio    522341 non-null float64
userSellerAction_3_ratio    522341 non-null float64
dtypes: float64(8), int64(2)
memory usage: 43.8 MB


In [55]:
 ###统计用户对预测商店点击的总天数
clickDays = logs[logs.action_type == 0]
clickDays = clickDays[["user_id", "seller_id","time_stamp","action_type"]]
clickDays = clickDays.groupby(['user_id', 'seller_id']).apply(lambda x:x.time_stamp.nunique()).reset_index()
clickDays.rename(columns={0:'click_days'},inplace=True)
df_result = pd.merge(df_result,clickDays,how='left',on=['user_id', 'seller_id'])
df_result.click_days.fillna(0,inplace=True)
del clickDays

In [56]:
###统计用户对预测商店加入购物车的总天数
addDays = logs[logs.action_type == 1]
addDays = addDays[["user_id", "seller_id","time_stamp","action_type"]]
addDays = addDays.groupby(['user_id', 'seller_id']).apply(lambda x:x.time_stamp.nunique()).reset_index()
addDays.rename(columns={0:'add_days'},inplace=True)
df_result = pd.merge(df_result,addDays,how='left',on=['user_id', 'seller_id'])
df_result.add_days.fillna(0,inplace=True)
del addDays

In [57]:
###统计用户对预测商店购物的总天数
buyDays = logs[logs.action_type == 2]
buyDays = buyDays[["user_id", "seller_id","time_stamp","action_type"]]
buyDays = buyDays.groupby(['user_id', 'seller_id']).apply(lambda x:x.time_stamp.nunique()).reset_index()
buyDays.rename(columns={0:'buy_days'},inplace=True)
df_result = pd.merge(df_result,buyDays,how='left',on=['user_id', 'seller_id'])
df_result.buy_days.fillna(0,inplace=True)
del buyDays

In [58]:
###统计用户对预测商店收藏的总天数
saveDays = logs[logs.action_type == 3]
saveDays = saveDays[["user_id", "seller_id","time_stamp","action_type"]]
saveDays = saveDays.groupby(['user_id', 'seller_id']).apply(lambda x:x.time_stamp.nunique()).reset_index()
saveDays.rename(columns={0:'save_days'},inplace=True)
df_result = pd.merge(df_result,saveDays,how='left',on=['user_id', 'seller_id'])
df_result.save_days.fillna(0,inplace=True)
del saveDays

In [59]:
itemCount = logs[["user_id", "seller_id","item_id","action_type"]]
# 点击商品数量
itemCountClick = itemCount[itemCount.action_type == 0]
item_result = itemCountClick.groupby(['user_id', 'seller_id']).apply(lambda x:x.item_id.nunique()).reset_index()
item_result.rename(columns={0:'item_click_count'},inplace=True)
item_result.item_click_count.fillna(0,inplace=True)
df_result = pd.merge(df_result,item_result,how='left',on=['user_id', 'seller_id'])
del itemCountClick,item_result
# 加入购物车商品数量
itemCountAdd = itemCount[itemCount.action_type == 1]
item_result = itemCountAdd.groupby(['user_id', 'seller_id']).apply(lambda x:x.item_id.nunique()).reset_index()
item_result.rename(columns={0:'item_add_count'},inplace=True)
item_result.item_add_count.fillna(0,inplace=True)
df_result = pd.merge(df_result,item_result,how='left',on=['user_id', 'seller_id'])
del itemCountAdd,item_result
# 购买商品数量
itemCountBuy = itemCount[itemCount.action_type == 2]
item_result = itemCountBuy.groupby(['user_id', 'seller_id']).apply(lambda x:x.item_id.nunique()).reset_index()
item_result.rename(columns={0:'item_buy_count'},inplace=True)
item_result.item_buy_count.fillna(0,inplace=True)
df_result = pd.merge(df_result,item_result,how='left',on=['user_id', 'seller_id'])
del itemCountBuy,item_result
# 收藏商品数量
itemCountSave = itemCount[itemCount.action_type == 3]
item_result = itemCountSave.groupby(['user_id', 'seller_id']).apply(lambda x:x.item_id.nunique()).reset_index()
item_result.rename(columns={0:'item_save_count'},inplace=True)
item_result.item_save_count.fillna(0,inplace=True)
df_result = pd.merge(df_result,item_result,how='left',on=['user_id', 'seller_id'])
del itemCountSave,item_result

In [60]:
catCount = logs[["user_id", "seller_id","cat_id","action_type"]]
# 点击种类数量
catCountClick = catCount[catCount.action_type == 0]
cat_result = catCountClick.groupby(['user_id', 'seller_id']).apply(lambda x:x.cat_id.nunique()).reset_index()
cat_result.rename(columns={0:'cat_click_count'},inplace=True)
cat_result.cat_click_count.fillna(0,inplace=True)
df_result = pd.merge(df_result,cat_result,how='left',on=['user_id', 'seller_id'])
del catCountClick,cat_result
# 加入购物车种类数量
catCountAdd = catCount[catCount.action_type == 1]
cat_result = catCountAdd.groupby(['user_id', 'seller_id']).apply(lambda x:x.cat_id.nunique()).reset_index()
cat_result.rename(columns={0:'cat_add_count'},inplace=True)
cat_result.cat_add_count.fillna(0,inplace=True)
df_result = pd.merge(df_result,cat_result,how='left',on=['user_id', 'seller_id'])
del catCountAdd,cat_result
# 购买种类数量
catCountBuy = catCount[catCount.action_type == 2]
cat_result = catCountBuy.groupby(['user_id', 'seller_id']).apply(lambda x:x.cat_id.nunique()).reset_index()
cat_result.rename(columns={0:'cat_buy_count'},inplace=True)
cat_result.cat_buy_count.fillna(0,inplace=True)
df_result = pd.merge(df_result,cat_result,how='left',on=['user_id', 'seller_id'])
del catCountBuy,cat_result
# 收藏种类数量
catCountSave = catCount[catCount.action_type == 3]
cat_result = catCountSave.groupby(['user_id', 'seller_id']).apply(lambda x:x.cat_id.nunique()).reset_index()
cat_result.rename(columns={0:'cat_save_count'},inplace=True)
cat_result.cat_save_count.fillna(0,inplace=True)
df_result = pd.merge(df_result,cat_result,how='left',on=['user_id', 'seller_id'])
del catCountSave,cat_result

In [61]:
brandCount = logs[["user_id", "seller_id","brand_id","action_type"]]
# 点击品牌数量
brandCountClick = brandCount[brandCount.action_type == 0]
brand_result = brandCountClick.groupby(['user_id', 'seller_id']).apply(lambda x:x.brand_id.nunique()).reset_index()
brand_result.rename(columns={0:'brand_click_count'},inplace=True)
brand_result.brand_click_count.fillna(0,inplace=True)
df_result = pd.merge(df_result,brand_result,how='left',on=['user_id', 'seller_id'])
del brandCountClick,brand_result
# 加入购物车品牌数量
brandCountAdd = brandCount[brandCount.action_type == 1]
brand_result = brandCountAdd.groupby(['user_id', 'seller_id']).apply(lambda x:x.brand_id.nunique()).reset_index()
brand_result.rename(columns={0:'brand_add_count'},inplace=True)
brand_result.brand_add_count.fillna(0,inplace=True)
df_result = pd.merge(df_result,brand_result,how='left',on=['user_id', 'seller_id'])
del brandCountAdd,brand_result
# 购买品牌数量
brandCountBuy = brandCount[brandCount.action_type == 2]
brand_result = brandCountBuy.groupby(['user_id', 'seller_id']).apply(lambda x:x.brand_id.nunique()).reset_index()
brand_result.rename(columns={0:'brand_buy_count'},inplace=True)
brand_result.brand_buy_count.fillna(0,inplace=True)
df_result = pd.merge(df_result,brand_result,how='left',on=['user_id', 'seller_id'])
del brandCountBuy,brand_result
# 收藏品牌数量
brandCountSave = brandCount[brandCount.action_type == 3]
brand_result = brandCountSave.groupby(['user_id', 'seller_id']).apply(lambda x:x.brand_id.nunique()).reset_index()
brand_result.rename(columns={0:'brand_save_count'},inplace=True)
brand_result.brand_save_count.fillna(0,inplace=True)
df_result = pd.merge(df_result,brand_result,how='left',on=['user_id', 'seller_id'])
del brandCountSave,brand_result

df_result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 522341 entries, 0 to 522340
Data columns (total 26 columns):
user_id                     522341 non-null int64
seller_id                   522341 non-null int64
userSellerAction_0          522341 non-null float64
userSellerAction_1          522341 non-null float64
userSellerAction_2          522341 non-null float64
userSellerAction_3          522341 non-null float64
userSellerAction            522341 non-null float64
userSellerAction_0_ratio    522341 non-null float64
userSellerAction_1_ratio    522341 non-null float64
userSellerAction_3_ratio    522341 non-null float64
click_days                  522341 non-null float64
add_days                    522341 non-null float64
buy_days                    522341 non-null int64
save_days                   522341 non-null float64
item_click_count            462933 non-null float64
item_add_count              9394 non-null float64
item_buy_count              522341 non-null int64
item_save_count

In [62]:
df_result.fillna(0,inplace=True)
# 对数值型特征手动标准化
for col in ['buy_days','item_buy_count','cat_buy_count','brand_buy_count']:
    df_result[col] = df_result[col].astype('float64')
df_result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 522341 entries, 0 to 522340
Data columns (total 26 columns):
user_id                     522341 non-null int64
seller_id                   522341 non-null int64
userSellerAction_0          522341 non-null float64
userSellerAction_1          522341 non-null float64
userSellerAction_2          522341 non-null float64
userSellerAction_3          522341 non-null float64
userSellerAction            522341 non-null float64
userSellerAction_0_ratio    522341 non-null float64
userSellerAction_1_ratio    522341 non-null float64
userSellerAction_3_ratio    522341 non-null float64
click_days                  522341 non-null float64
add_days                    522341 non-null float64
buy_days                    522341 non-null float64
save_days                   522341 non-null float64
item_click_count            522341 non-null float64
item_add_count              522341 non-null float64
item_buy_count              522341 non-null float64
item_save

In [63]:
# 对数值型特征手动标准化
numeric_cols = df_result.columns[df_result.dtypes == 'float64']
print(numeric_cols)
numeric_col_means = df_result.loc[:, numeric_cols].mean()
numeric_col_std = df_result.loc[:, numeric_cols].std()
df_result.loc[:, numeric_cols] = (df_result.loc[:, numeric_cols] - numeric_col_means) / numeric_col_std
df_result.head(5)
#df_result.info()
#df_result.drop(columns=['buy_days'])
#df_result.info()

Index(['userSellerAction_0', 'userSellerAction_1', 'userSellerAction_2',
       'userSellerAction_3', 'userSellerAction', 'userSellerAction_0_ratio',
       'userSellerAction_1_ratio', 'userSellerAction_3_ratio', 'click_days',
       'add_days', 'buy_days', 'save_days', 'item_click_count',
       'item_add_count', 'item_buy_count', 'item_save_count',
       'cat_click_count', 'cat_add_count', 'cat_buy_count', 'cat_save_count',
       'brand_click_count', 'brand_add_count', 'brand_buy_count',
       'brand_save_count'],
      dtype='object')


,user_id,seller_id,userSellerAction_0,userSellerAction_1,userSellerAction_2,userSellerAction_3,userSellerAction,userSellerAction_0_ratio,userSellerAction_1_ratio,userSellerAction_3_ratio,...,item_buy_count,item_save_count,cat_click_count,cat_add_count,cat_buy_count,cat_save_count,brand_click_count,brand_add_count,brand_buy_count,brand_save_count
0,1,1019,0.048748,-0.113497,3.113534,-0.264393,0.162225,0.115018,2.923576,2.042316,...,-0.348307,-0.271276,-0.376071,-0.125234,-0.293369,-0.368967,0.096026,-0.135321,-0.089934,-0.466480
1,2,1679,-0.487477,-0.113497,1.944228,-0.264393,-0.402599,2.287236,2.123655,1.578144,...,1.028622,-0.271276,-0.897370,-0.125234,-0.293369,-0.368967,-1.817830,-0.135321,-0.089934,-0.466480
2,2,1784,-0.487477,-0.113497,0.774921,-0.264393,-0.453946,1.999889,1.092378,0.979721,...,-0.348307,-0.271276,-0.897370,-0.125234,-0.293369,-0.368967,-1.817830,-0.135321,-0.089934,-0.466480
3,3,2313,-0.272987,-0.113497,-0.394386,0.416284,-0.248556,-0.012666,-0.361126,-1.305560,...,-0.348307,0.465957,0.145228,-0.125234,-0.293369,1.076768,0.096026,-0.135321,-0.089934,2.007016
4,4,1186,-0.272987,-0.113497,-0.394386,-0.264393,-0.299903,-0.012666,-0.361126,0.136292,...,-0.348307,-0.271276,-0.376071,-0.125234,-0.293369,-0.368967,0.096026,-0.135321,-0.089934,-0.466480


In [65]:
df_result.drop("buy_days",axis=1,inplace=True)
df_result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 522341 entries, 0 to 522340
Data columns (total 25 columns):
user_id                     522341 non-null int64
seller_id                   522341 non-null int64
userSellerAction_0          522341 non-null float64
userSellerAction_1          522341 non-null float64
userSellerAction_2          522341 non-null float64
userSellerAction_3          522341 non-null float64
userSellerAction            522341 non-null float64
userSellerAction_0_ratio    522341 non-null float64
userSellerAction_1_ratio    522341 non-null float64
userSellerAction_3_ratio    522341 non-null float64
click_days                  522341 non-null float64
add_days                    522341 non-null float64
save_days                   522341 non-null float64
item_click_count            522341 non-null float64
item_add_count              522341 non-null float64
item_buy_count              522341 non-null float64
item_save_count             522341 non-null float64
cat_click

In [64]:
#保存用户特征
filePath='D:\\数据挖掘\data_format1\\userSellerInfo_Features_3.pkl'
pickle.dump(df_result, open(filePath, 'wb'))
df_result.to_csv('D:\\数据挖掘\data_format1\\user_seller_info_format_feature_3.csv')